In [42]:
# import everything we may need
import mysql.connector
from mysql.connector import errorcode
import config
import requests
import time
import json
import re
import pickle
from datetime import datetime

In [2]:
#get data from box office mojo as json file
f = open('mojo_data.json')
data = json.load(f)

In [14]:
# connect to AWS
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password
)

# Establish cursor
cursor = cnx.cursor()

## function to close the connection
def close_connections():
    cursor.close()
    conn.close()

In [ ]:
#name the db
db_name = 'Mod_1_Project'

In [ ]:
# create database in AWS
def create_database(cursor, database_name):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database_name))
    except mysql.connector.Error as err:
        ## Catch the error if an error occurs.
        print("Failed creating database: {}".format(err))
        exit(1)
        
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)

In [88]:
#get list of movies from box office mojo data
titles = [x['title'] for x in data]
titles

2050

In [4]:
#function that splits titles from BOM 
#that come with years so that we can use 
#the list to call OMDB w/out the title
def find_year_in_title(title):
    year = re.findall('(201\d)', title)
    if year:
        return (title.replace("("+year[0]+")", ""), year[0])
    else:
        return title

list_of_movie_titles = []
for title in titles:
    list_of_movie_titles.append(find_year_in_title(title))
    
list_of_movie_titles

["Tyler Perry's Boo 2! A Madea Halloween",
 "Daddy's Home 2",
 'Everything, Everything',
 'Monster Trucks',
 'How to be a Latin Lover',
 'The Nut Job 2:Nutty by Nature',
 'American Assassin',
 'Annabelle:Creation',
 'Star Wars:The Last Jedi',
 'Snatched',
 'Baby Driver',
 'Resident Evil:The Final Chapter',
 'Get Out',
 'Ferdinand',
 'Kong:Skull Island',
 'Pitch Perfect 3',
 'Rough Night',
 'Transformers:The Last Knight',
 'Fist Fight',
 'The Dark Tower',
 'American Made',
 'Underworld:Blood Wars',
 'Darkest Hour',
 'The Foreigner',
 'Home Again',
 "A Dog's Purpose",
 "The Hitman's Bodyguard",
 'Justice League',
 ('Life ', '2017'),
 'Rings',
 'The Fate of the Furious',
 'The Mountain Between Us',
 ('Murder on the Orient Express ', '2017'),
 'King Arthur:Legend of the Sword',
 'Smurfs:The Lost Village',
 'Geostorm',
 'Dunkirk',
 'Pirates of the Caribbean:Dead Men Tell No Tales',
 'Valerian and the City of a Thousand Planets',
 ('Going in Style ', '2017'),
 ('Power Rangers ', '2017'),
 'A

In [115]:
key = config.apiKey
url = 'http://www.omdbapi.com/?i=tt3896198&apikey='

# Make call to API
# Iterate over list of titles (from box office mojo) to retrieve all

def OMDB_call(titles):
    all_titles = []
    for title in titles:
        if type(title) == tuple:
            parameters = {'t': title[0], 'y': title[1]}
        else:
            parameters = {'t': title}
        response = requests.get(url + key, params=parameters)    
        data = response.json()
        data['bom_title'] = title
        all_titles.append(data)
        time.sleep(.5)
        # show progress
        if titles.index(title) % 50 == 0:
            print("progress:", titles.index(title),"/", len(titles))
            filename = 'all_data'
            outfile = open(filename, 'wb')
            pickle.dump(all_titles,outfile)
            outfile.close()
    return all_titles

# movie_dict = OMDB_call(list_of_movie_titles)
len(movie_dict)

2050

In [116]:
#function checking which movies didn't return anything after the call
# missing_movies = set(titles)-set([x['bom_title'] for x in movie_dict if 'Title' in x.keys()])
# movie_dict
#pickle missing titles for record
import pickle
filename = 'all_date'
outfile = open(filename, 'wb')
pickle.dump(movie_dict,outfile)
outfile.close()

#function to access pickled file
def access_pickle(filename):
    infile = open(filename,'rb')
    new_dict = pickle.load(infile)
    infile.close()

In [103]:
#function that checks if list from BOM has returned anything from OMDB
def check_for_title(data):
    movies_dict = []
    for movie in movie_dict:
        if movie['Response'] == 'False':
            movies_dict.append(movie)
    return movies_dict
movies_dict = check_for_title(movie_dict)

len(movies_dict)

1211

In [84]:
#function to convert date string into date type that SQL expects (0000/00/00)
def string_to_time(string):
    return datetime.strftime(datetime.strptime(string, "%d %b %Y"), "%Y-%m-%d")

# iterate over movie dictinoary and convert string to time
# for movie in movies_dict:
#     movie['Released'] = string_to_time(movie['Released'])

movies_dict


[{'Title': 'Berlin Syndrome',
  'Year': '2017',
  'Rated': 'R',
  'Released': '26 May 2017',
  'Runtime': '116 min',
  'Genre': 'Drama, Horror, Mystery, Thriller',
  'Director': 'Cate Shortland',
  'Writer': 'Shaun Grant (screenplay), Melanie Joosten (novel), Cate Shortland (additional material)',
  'Actors': 'Teresa Palmer, Max Riemelt, Matthias Habich, Emma Bading',
  'Plot': 'A passionate holiday romance leads to an obsessive relationship, when an Australian photojournalist wakes one morning in a Berlin apartment and is unable to leave.',
  'Language': 'English, German, Russian',
  'Country': 'Australia',
  'Awards': '13 nominations.',
  'Poster': 'https://m.media-amazon.com/images/M/MV5BYzQ0YWRjYjctYmQ5ZC00M2M2LWE0NzMtMTI4ZDg1NjIyZTM1XkEyXkFqcGdeQXVyMTM2MzgyOTU@._V1_SX300.jpg',
  'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.3/10'},
   {'Source': 'Rotten Tomatoes', 'Value': '74%'},
   {'Source': 'Metacritic', 'Value': '70/100'}],
  'Metascore': '70',
  'imdbRating':

In [85]:
#convert box office string to integer
def str_to_int(movies_dict):
    try:
        for movie in movies_dict:
            if type(movie['BoxOffice']) != int:
                if movie['BoxOffice'] != 'N/A':
                    movie['BoxOffice'] = int(movie['BoxOffice'][1:].replace(",", ""))
    except KeyError:
        print(movie)
str_to_int(movies_dict)


{'Title': 'The Glory of Tang Dynasty', 'Year': '2017–', 'Rated': 'N/A', 'Released': '29 Jan 2017', 'Runtime': 'N/A', 'Genre': 'Drama, History, Romance', 'Director': 'N/A', 'Writer': 'N/A', 'Actors': 'Tian Jing, Jialun Ren, Han Chin, Yuan Du', 'Plot': 'The series is about the ups and downs in the life of Shen Zhen Zhu, a exceptional beauty living during the most turbulent periods of the Tang dynasty.', 'Language': 'Chinese', 'Country': 'China', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BNTI2MTZkYmEtNTMyZC00MWU5LTk1OGItYmQzZjlkYzZiMzE2L2ltYWdlXkEyXkFqcGdeQXVyNTAwNzc3ODg@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.2/10'}], 'Metascore': 'N/A', 'imdbRating': '7.2', 'imdbVotes': '38', 'imdbID': 'tt6492378', 'Type': 'series', 'totalSeasons': 'N/A', 'Response': 'True'}


In [25]:
#function to create tables in our database
def create_table(dict_of_tables, db_name):
    for table_name in dict_of_tables:
        table_query = dict_of_tables[table_name]
        cursor.execute("USE {}".format(db_name))
        try:
            print("Creating table {}: ".format(table_name), end='')
            cursor.execute(table_query)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
                print("already exists.")
            else:
                print(err.msg)
        else:
            print("OK")

In [49]:
#Create tables to input data
TABLES = {}
TABLES['main_movie_table'] = (
    "CREATE TABLE main_movie_table ("
    "  movie_id int NOT NULL AUTO_INCREMENT,"
    "  omdb_title text,"
    "  bom_title text"
    "  budget int(10),"
    "  box_office int(15)"
    "  release_date date,"
    "  award_wins int(3),"
    "  award_nominations int(3),"
    "  RT_rating int(2),"
    "  IMDB_rating real,"
    "  PRIMARY KEY (movie_id)"
    ") ENGINE=InnoDB")

TABLES['genres'] = (
    "CREATE TABLE genres ("
    "  genre_id int NOT NULL AUTO_INCREMENT,"
    "  genre text,"
    "  PRIMARY KEY (genre_id)"
    ") ENGINE=InnoDB")

TABLES['key_words'] = (
    "CREATE TABLE key_words ("
    "  keyword_id int NOT NULL AUTO_INCREMENT,"
    "  key_words text,"
    "  PRIMARY KEY (keyword_id)"
    ") ENGINE=InnoDB")

TABLES['movies_genres'] = (
    "CREATE TABLE movies_genres ("
      " movie_id int NOT NULL,"
      " genre_id int NOT NULL"
      ") ENGINE=InnoDB")

TABLES['movies_keywords'] = (
    "CREATE TABLE movies_keywords ("
      " movie_id int NOT NULL,"
      " keyword_id text NOT NULL"
      ") ENGINE=InnoDB")

TABLES['actors'] = (
    "CREATE TABLE actors ("
    " actor_id int NOT NULL AUTO_INCREMENT," 
    " actor_name text NOT NULL,"
    " PRIMARY KEY (actor_id)"
    ") ENGINE=InnoDB")

TABLES['movie_actors'] = (
    "CREATE TABLE movie_actors("
    " movie_id int NOT NULL,"
    " actor_id int NOT NULL"
    ") ENGINE=InnoDB")

create_table(TABLES, 'Mod_1_Project')

Creating table main_movie_table: OK
Creating table genres: already exists.
Creating table key_words: already exists.
Creating table movies_genres: already exists.
Creating table movies_keywords: already exists.
Creating table actors: already exists.
Creating table movie_actors: already exists.


In [86]:
# pickle our first 445 titles from OMDB
# filename = 'first_445'
# outfile = open(filename, 'wb')
# pickle.dump(movies_dict,outfile)
# outfile.close()

#function to access pickled file
def access_pickle(filename):
    infile = open(filename,'rb')
    new_dict = pickle.load(infile)
    infile.close()

#pickle our next 200 titles from OMDB
filename = '445_to_665'
outfile = open(filename, 'wb')
pickle.dump(movies_dict,outfile)
outfile.close()

In [50]:
#insert some of the movie data into tables
def add_movies_toDB(movies_dict):
    for movie in movies_dict:
        try:
            cursor.execute('''USE Mod_1_Project''')
            cursor.execute('''insert into main_movie_table 
            (title, release_date, RT_rating, IMDB_rating) values(%s,%s,%s,%s)''',
            (movie['Title'], movie['Released'], movie['Ratings'][1]['Value'].replace('%', ''), movie['imdbRating'])) 
            cnx.commit()
        except IndexError:
            cursor.execute('''USE Mod_1_Project''')
            cursor.execute('''insert into main_movie_table 
            (title, release_date, IMDB_rating) values(%s,%s,%s)''',
            (movie['Title'], movie['Released'], movie['imdbRating'])) 
            print(movie)
            cnx.commit()
            

In [51]:
add_movies_toDB(movies_dict)

{'Title': 'The Oscar Nominated Short Films 2017: Animation', 'Year': '2017', 'Rated': 'PG-13', 'Released': '2017-02-08', 'Runtime': '88 min', 'Genre': 'Animation, Short', 'Director': 'N/A', 'Writer': 'N/A', 'Actors': 'Nicki Bluhm, Caroline Dhavernas, Greg Dykstra', 'Plot': 'A collection of the animated short films nominated for the 2017 Academy Awards.', 'Language': 'N/A', 'Country': 'USA', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZDNhOWIwMjUtYmE4NS00MTkxLTgzZjYtYTcwMGJmZDgyZTRkXkEyXkFqcGdeQXVyNjU2NTMxOTg@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.3/10'}], 'Metascore': 'N/A', 'imdbRating': '7.3', 'imdbVotes': '185', 'imdbID': 'tt6511760', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True'}
{'Title': 'In Our Hands: The Battle for Jerusalem', 'Year': '2017', 'Rated': 'NOT RATED', 'Released': '2017-05-23', 'Runtime': '109 min', 'Genre': 'Documentary', 'Director': 'Erin 

{'Title': 'Sword Art Online: The Movie - Ordinal Scale', 'Year': '2017', 'Rated': 'TV-14', 'Released': '2017-02-18', 'Runtime': '119 min', 'Genre': 'Animation, Action, Adventure, Fantasy, Sci-Fi', 'Director': 'Tomohiko Itô', 'Writer': 'Tomohiko Itô (screenplay), Reki Kawahara (characters), Reki Kawahara (screenplay)', 'Actors': 'Yoshitsugu Matsuoka, Haruka Tomatsu, Kanae Itô, Ayana Taketatsu', 'Plot': 'Kirito uncovers a conspiracy within Ordinal Scale, a popular VR game developed for a new system called The Augma.', 'Language': 'Japanese', 'Country': 'Japan', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZGE4M2M2OTYtZDgwMy00NGYwLWE0YTYtYzkyMDFjYWFhN2QzXkEyXkFqcGdeQXVyMzgxODM4NjM@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.4/10'}], 'Metascore': 'N/A', 'imdbRating': '7.4', 'imdbVotes': '4,621', 'imdbID': 'tt5544384', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True'}
{'Titl

{'Title': 'Chris Brown: Welcome to My Life', 'Year': '2017', 'Rated': 'Unrated', 'Released': '2017-06-08', 'Runtime': '80 min', 'Genre': 'Documentary', 'Director': 'Andrew Sandler', 'Writer': 'N/A', 'Actors': 'Mary J. Blige, Chris Brown, DJ Khaled, Jamie Foxx', 'Plot': "A compelling and never-before-scene look at Chris Brown's controversial life and career.", 'Language': 'English', 'Country': 'USA', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BZGEzNjFiNzItNzE4Mi00YzU0LWE2ZWEtYzBhOTE3NjY2M2IyXkEyXkFqcGdeQXVyNDA2ODEzOTQ@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.8/10'}], 'Metascore': 'N/A', 'imdbRating': '5.8', 'imdbVotes': '895', 'imdbID': 'tt5588850', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'Riveting Entertainment', 'Website': 'http://www.chrisbrowndocumentary.com/', 'Response': 'True'}
{'Title': 'Sailor Moon R: The Movie: The Promise of the Rose', 'Year': '1993', 'Rated': 'TV-14', 'Released': '2000-02-

{'Title': 'A Manifesto for the Un-communal', 'Year': '2017', 'Rated': 'N/A', 'Released': '2017-08-06', 'Runtime': '11 min', 'Genre': 'Documentary, Short', 'Director': 'Syllas Tzoumerkas', 'Writer': 'Syllas Tzoumerkas', 'Actors': 'Maria Filini, Youla Boudali, Angeliki Papoulia, Syllas Tzoumerkas', 'Plot': 'N/A', 'Language': 'English, Greek, Hebrew', 'Country': 'Germany, Israel', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BYWYwNzMzM2ItMzUzOS00OWUxLThkNDAtNTVlYTY3NDRkNjhjXkEyXkFqcGdeQXVyNTM3MzQ4NDk@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.2/10'}], 'Metascore': 'N/A', 'imdbRating': '7.2', 'imdbVotes': '9', 'imdbID': 'tt7132820', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True'}
{'Title': 'Valley of Bones', 'Year': '2017', 'Rated': 'R', 'Released': '2017-09-01', 'Runtime': '90 min', 'Genre': 'Crime, Thriller, Western', 'Director': 'Dan Glaser', 'Writer': 'Dan Glaser, Ste

{'Title': 'We Love You, Sally Carmichael!', 'Year': '2017', 'Rated': 'PG', 'Released': '2017-08-04', 'Runtime': '84 min', 'Genre': 'Comedy', 'Director': 'Christopher Gorham', 'Writer': 'Daryn Tufts', 'Actors': 'Felicia Day, Christopher Gorham, Paula Marshall, Elizabeth Tulloch', 'Plot': "Readers across the world are in love with author Sally Carmichael's series of romance novels that chronicle the epic love story between a human girl and a merman. But no one knows that ...", 'Language': 'N/A', 'Country': 'USA', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BYjFiYjE0MzQtNjU0Zi00MDJjLTgwNDEtN2RiMzNkZTM1OGE1XkEyXkFqcGdeQXVyMTc0MTAwMg@@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '6.0/10'}], 'Metascore': 'N/A', 'imdbRating': '6.0', 'imdbVotes': '289', 'imdbID': 'tt5664676', 'Type': 'movie', 'DVD': '19 Dec 2017', 'BoxOffice': '$28,500', 'Production': 'GO Films', 'Website': 'N/A', 'Response': 'True'}
{'Title': 'The Journey', 'Year': '1959',

{'Title': 'Twenty Two', 'Year': '2015', 'Rated': 'N/A', 'Released': '2017-09-08', 'Runtime': '112 min', 'Genre': 'Documentary, Crime, History, War', 'Director': 'Ke Guo', 'Writer': 'N/A', 'Actors': 'N/A', 'Plot': 'An estimated 200,000 Chinese women were forced into prostitution by the Japanese army during WWII. Only 22 of them remain today to speak out publicly. This documentary is not a film for ...', 'Language': 'Mandarin, Chinese', 'Country': 'China', 'Awards': 'N/A', 'Poster': 'https://m.media-amazon.com/images/M/MV5BYWZjNWI5OTYtZDEwYy00YTY5LTllNGMtYzJiMDNmOTM3NGY1XkEyXkFqcGdeQXVyNjc2MDYyODM@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.6/10'}], 'Metascore': 'N/A', 'imdbRating': '7.6', 'imdbVotes': '279', 'imdbID': 'tt3843282', 'Type': 'movie', 'DVD': 'N/A', 'BoxOffice': 'N/A', 'Production': 'N/A', 'Website': 'N/A', 'Response': 'True'}
{'Title': "The King's Case Note", 'Year': '2017', 'Rated': 'N/A', 'Released': '2017-04-26', 'Runtime': '114 min', '

{'Title': 'Beautiful Accident', 'Year': '2017', 'Rated': 'N/A', 'Released': '2017-05-26', 'Runtime': '104 min', 'Genre': 'Comedy, Fantasy', 'Director': 'Wi Ding Ho', 'Writer': 'Zhichao Ha (adapted screenplay), Je-Young Kim (original screenplay), Harry Segall (based on the play written by), Jialin Zhou (adapted screenplay)', 'Actors': 'Lun-Mei Kwei, Jingchun Wang, Na-Na OuYang, William Wang', 'Plot': 'A female lawyer had lived a very apathetic career until she had a terrible car accident. She died in the accident but when she reached the afterlife she learned that she came earlier than ...', 'Language': 'Mandarin', 'Country': 'China', 'Awards': 'N/A', 'Poster': 'https://images-na.ssl-images-amazon.com/images/M/MV5BOGU1Y2VlMTItYTdlNS00ZTlkLTkxNGEtNDViMGNhYWY3YWVhXkEyXkFqcGdeQXVyNzI1NzMxNzM@._V1_SX300.jpg', 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '7.0/10'}], 'Metascore': 'N/A', 'imdbRating': '7.0', 'imdbVotes': '93', 'imdbID': 'tt6797204', 'Type': 'movie', 'DVD': 'N/A',

In [ ]:
#insert the genre data into genre table         
def add_genres_toDB():
    for movie in movies_dict:
        cursor.execute('''USE Mod_1_Project''')
        if movie['Genre']
        ('''insert into genres (genre) values(%s)''', movie['Genre'])
        conn.commit

In [90]:
#len((set(titles)-set([x['Title'] for x in movies_dict 
#if 'Title' in x.keys()]))-(set([x['Title'] 
#for x in movies_dict if 'Title' in x.keys()])-set(titles)))




1906